This notebook builds on `Preprocessing.ipynb' and uses the data to construct surrogate models that can be used for controller optimization studies.



#  Surrogate Model for Controller Gains Optimization

Models that balance accuracy against computational costs are advantageous when designing floating offshore wind turbines (FOWT) using OpenFAST as the high-fidelity model through optimization studies, as several hundred predictive function evaluations might be necessary to identify the optimal solution. 

OpenFAST models the dynamic response of FOWT systems as a system of differential-algebraic equations (DAE).
Whenever OpenFAST is evaluated for a load case, this DAE system is solved to obtain the dynamic response of the FOWT. OpenFAST also uses different modules like AeroDyn, Hydrodyn etc. to capture the appropriate physics.
Therefore evaluating OpenFAST can be computationally expensive, making it unsuitable for optimization based studies where several hundred model evaluations might be required.

## Surrogate Model Requirements

Consider the following figure illustrating the steps involved in a controller optimization study:

<img src="EAB-copt.svg" width="600"/>

The goal is to develop a surrogate model to replace the aero-hydro-servo-elastic model that can predict the timeseries of outputs given the inputs and controls. Once constructed, this model can then be coupled with the controller as shown in the figure:

<img src="SM2.svg" width="800"/>


There are different approaches that can be used to construct such a surrogate model such as classic system identification (sys-id) methods and long-short-term memory (LSTM) networks.
These methods can be classified as 'black-box' approaches to map the inputs to the outputs. They have been widely utilized for various complex engineering systems.

However, in this example we use the derivative function surrogate modeling (DFSM) approach to construct the surrogate model.

## Derivative Function Surrogate Model (DFSM)

DFSM are surrogate models of the dynamic function or derivative function, that describes how the state derivatives evolve for the given system.
A key assumption that is made when using DFSM approach is that the system dynamics can be described by an ordinary differential equation (ODE) of the following form:
    \begin{align}
            \dot{\xi} &= f(\xi,u)\\
            y &= g(\xi,u)
    \end{align}
Where $\xi$ denotes the states, $u$ denotes the controls, $y$ denotes the outputs, and $f$, $g$ correspond to the state derivative and output functions respectively.      

DFSM approximates $f$ and $g$ as:
    \begin{align}
        \dot{\xi} &\approx f_{\text{DFSM}}(\xi,u)\\
        y &\approx g_{\text{DFSM}}(\xi,u) \\
    \end{align}

Unlike the aforementioned black-box approaches, the DFSM approach allows the user to prescribe a relation between the quantities approximated. In this study, we assume that the underlying ODE can be approximated as an linear-parameter varying (LPV) system, where the key parameter is the current speed ($v$):
    \begin{align}
        f_{\text{DFSM}}(\xi,u) = A(v)\xi + B(v)u\\
        g_{\text{DFSM}}(\xi,u) = C(v)\xi + D(v)u \\
    \end{align}


## Model Construction

### Inputs
The inputs when constructing the DFSM are turbulent OpenFAST simulations for different current speeds. More details regarding these inputs, and how they are preprocessed can be found in `Preprocessing.ipynb'.

Once the inputs are available and preprocessed, then the LPV model is constructed as follows:
1. Individual state-space matrices $\Sigma = [A,B,C,D]$ are obtained for different current speeds values $v$
2. The system matrices $\Sigma$ are then linearly interpolated over $v$ to obtain a continuous model. For more details regarding LPV modeling, please refer to [https://doi.org/10.1115/1.4063969](https://doi.org/10.1115/1.4063969) 

### Obtaining linear models
We solve an optimization problem to obtain the system matrices $\Sigma$. The $A$ and $B$ matrices are evaluated as part of one problem, and the $C$ and $D$ matrices are evaluate separately, as they predict different quantities, and have different requirements.


For each current speed, an optimization problem is solved to identify the model parameters ($\theta$) for the A and B matrices, and can be formulated as:
\begin{align}
    \min_{\theta} \frac{Tr(E'*E)}{N} \\
    \text{s.t} \quad max(real(eig(A(\theta))) < -\delta\\
    \text{where:} \quad E = \dot{\xi}_{\text{act}} - \dot{\xi}_{\text{DFSM}}\\
                     \dot{\xi}_{\text{DFSM}} = A(\theta)\xi + B(\theta)u  \\
                     Tr~ \text{is the trace operation}
\end{align}
The constraint on the eigen value is required to ensure the the simulations and the state-space system remain stable.
We use a hybrid optimization strategy to solve this problem. A genetic algorithm is used to get a good initial estimate of $\theta$, then a gradient-based optimizer using the interior-point algorithm is used to solve the problem. 
The python script 'construct_DFSM.py' will formulate and solve this problem. The rest of this notebook can be considered supplementary material to explain how the problem is constructed and solved, and also offers some hueristics to setup the problem.


## construct_DFSM.py Requirements
The script `construct_DFSM.py' constructs and solves the optimization problem used in constructing the DFSM. The main requirement to run this script is the matlab api required to call matlab from python, and the OpenFAST simulation data. 

### Matlab API
Currently, this script uses the genetric algorithm solver, and the nonlinear optimizer fmincon in matlab to solve the optimization problem.
To do this, we use the matlab api to interface with these optimizers. Please refer to the following link on how to install the matlab API: [https://www.mathworks.com/help/matlab/matlab-engine-for-python.html](https://www.mathworks.com/help/matlab/matlab-engine-for-python.html)

### Compatability
It is important to note that matlab version 2024b is required for python version 3.12. More details regarding the compatability can be found in [https://www.mathworks.com/support/requirements/python-compatibility.html](https://www.mathworks.com/support/requirements/python-compatibility.html). 



## Inputs
Other than the matlab api, the main requirements to this file are the openfast simulations, and its details.
The rest of this notebook uses the simulations results available in the following directory: 
[https://colostate-my.sharepoint.com/:f:/g/personal/athulsun_colostate_edu/EqE_WXKZXedDra_j38rAkGEBuOxB1gv7WjpFgqNEvd4ypQ?e=rB9lIb](https://colostate-my.sharepoint.com/:f:/g/personal/athulsun_colostate_edu/EqE_WXKZXedDra_j38rAkGEBuOxB1gv7WjpFgqNEvd4ypQ?e=rB9lIb)

The folder contains two subfolders RM1_train, and RM1_test, which will be used in training and testing the the DFSM respectively.

For the simulations in these folders, three degrees of freedom we enabled, namely the Ptfmpitch, PtfmHeave, and GenSpeed. So these three quantities, along with their first derivatives will be states $\xi$ as part of the DFSM model.

The controls $u$ will be the rotor average wind speed 'RtVAvgxh', generator torque 'GenTq', blade pitch 'BldPitch1', and the wave elevation 'Wave1Elev'.

The outputs $y$ considered are tower-base fore-aft force 'TwrBsFxt', tower-base fore-aft moment 'TwrBsMyt', tower-top translational acceleration 'YawBrTAxp', tower-top rotational acceleration 'NcIMURAys', generator power 'GenPwr', fluid Cp and Ct 'RtFldCp','RtFldCt'.

The generator speed for the considered turbine model is two orders of magnitude higher than the other two states. This quantity is scaled so that it does not adversely affect the results of the optimization problem.


## Parameter Estimation
The OpenFAST simulations are preproceesed as discussed in `Preprocessing.ipynb'. Suppose we are working with 3 current speeds $c = (c_1,c_2,c_3)$, with 5 simulations each, the preprocessed data is organized accordingly:
\begin{align*}
    \begin{bmatrix} c_1^1 &c_2^1 & c_3^1 \\ c_1^2 &c_2^2 & c_3^2 \\ c_1^3 &c_2^3 & c_3^3 \\c_1^4 &c_2^4 & c_3^4 \\c_1^5 &c_2^5 & c_3^5 \end{bmatrix}
\end{align*}

For each current speed, the optimization problem outlined above is solved to obtain the system matrics $[A_{c_1},B_{c_1},C_{c_1},D_{c_1}]$, and these matrices are interpolated over $c$.


1. Suppose the optimization starts for current speed $c_1$, a genetic algorithm is used to identify a good starting point, and then the interior point solver is used to identify a local minima.
2. Then the optimization proceeds to identify the system matrices for current speed $c_2$. For $c_2$, the genetic algorithm is not used to identify a good starting point, instead the interior-point is solver is started directly with the parameters identified for $c_1$ as the starting point. This helps in minimizing the total model construction time.
3. Since the model identified for $c_1$ satisfies the constraints on the eigen value, the interior-point solver can efficiently find the optimal model parameters.
4. Subsequently, the model parameters identified for $c_2$ are used for $c_3$.
5. For this floating system, the entries in the $C,D$ matrices for the NcIMURAys quantity is approximated with the $A,B$ matrix entries for the second time derivative of the platform pitch.



## Huersitics 
Although this process is straight forward, the accuracy and stability of the constructed model depends on a few hyperparameters, namely:
1. The current speed from which the process is started
2. The parameter $\delta$ which is the upper limit on the real part of the eigen value
3. The number of simulations per current speed used in the optimization problem

Some tips to select the value of these hyperparameters are as follows:
1. Starting from a near rated current speed usually results in an accurate model
2. Typically, when the quantities are properly sclaed, setting $\delta = 0$ results in an accurate model. But sometimes, when setting $\delta = 0$, and if the constraint tolerence is set to $1e-4$, the optimizer can find a solution with the maximium real part to be $-1e-6$ which would satisfy the constraint, but result in a model with undamped response. In these cases, setting $\delta$ to be a small value $ \delta \leq 0.1$ would solve the issue.
3. Using 5 simulations for each current speed, usually results in an accurate model.

The dfsm included in the folder 19_DFSM was obtained by setting w_start to 2.4, and setting $\delta = 0$. The user is encouraged to play around with these hyperparameters to identify an accurate surrogate for their use case. 

## Validation

### Open-loop validation
1. To get an idea if the constructed surrogate model is accurate, an 'open-loop' validation test is run, where the model is simulated for a set of control signal from a test simulation, and the resulting states and outputs are compared to the OpenFAST simulation. 
2. The results of this test is plotted to the folder 'outputs/open-loop-validation'.
3. This test should give you an approximate idea on the construction process results in an accurate and stable model.
4. Some key states and outputs like GenSpeed, NcIMURAys, YarBrTAxp are important, as these quantities are the main feedback variables for closed-loop simulation.
5. A high degree of accuracy is required for GenSpeed.
6. For the other quantities it is important to check if the response is stable. Since these are second time derivatives, they tend to be noisy, and a high deal of accuracy can be hard.
7. If the results are unsatisfactory, then play around with the hyperparameters mentioned previously to get a good model.



### Closed-loop validation
1. Once the user finds the results of this test to be satisfactory, they are encouraged to run the script 'run_CL_validation.py', which runs a closed-loop validation test, where the DFSM is used as the 'plant' in a closed-loop simulation with the ROSCO controller.
2. The simulations available in the folder 'RM1_test' are used for this validation study.
3. This scipt is MPI parallelizable, and plots the time-series signals of the closed-loop simulation, including the controls, states, and outputs obtained using the DFSM and OpenFAST along with the power spectral density (PSD) of key quantities to the folder 'outputs/closed-loop-validation'.
4. The results from this test will give the user a good idea if the DFSM can be effectively used for closed-loop simulation, and controller optimization studies.
5. In addition to the quantities outlined in the open-loop validation study, comparing the two control signals, BldPitch1 and GenTq, and how testing how close they are to the OpenFAST values is the main focus of this test
6. The posprocessing logic from WEIS is used to postprocess these results and obtain the summary statistics, DELs